In [1]:
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

from IPython.display import clear_output
clear_output()
print("\nDone")


Done


In [2]:
%cd /content
!rm -rf CodeFormer
!git clone https://github.com/sczhou/CodeFormer.git
%cd /content/CodeFormer

/content
Cloning into 'CodeFormer'...
remote: Enumerating objects: 594, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 594 (delta 176), reused 167 (delta 147), pack-reused 349
Receiving objects: 100% (594/594), 17.31 MiB | 35.02 MiB/s, done.
Resolving deltas: 100% (287/287), done.
/content/CodeFormer


In [4]:
%cd /content/CodeFormer
!python basicsr/setup.py develop

/content/CodeFormer
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issue

In [5]:
%cd /content
!mkdir output
!mkdir input

%cd /content/input
from google.colab import files
from io import BytesIO
uploaded = files.upload()

/content
/content/input


Saving newin.mp4 to newin.mp4
Saving sam-audio.wav to sam-audio.wav


In [6]:
%cd /content
!rm -rf output
!mkdir output
!rm -rf /content/Wav2Lip/temp/result.avi
!rm -rf /content/Wav2Lip/temp/temp.wav
!rm -rf /content/Wav2Lip/results/result_voice.mp4
!rm -rf images
!mkdir images
%cd /content/CodeFormer
!rm -rf results
!mkdir results

/content
/content/CodeFormer


In [7]:
# checkpoints wav2lip.pth wav2lip_gan.pt
%cd /content/Wav2Lip
!python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '/content/input/newin.mp4' --audio '/content/input/sam-audio.wav'

/content/Wav2Lip
Using cuda for inference.
Reading video frames...
Number of frames available for inference: 227
/content/Wav2Lip/audio.py:100: FutureWarning: Pass sr=16000, n_fft=800 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,
(80, 1601)
Length of mel chunks: 497
  0% 0/4 [00:00<?, ?it/s]
  0% 0/15 [00:00<?, ?it/s]
  7% 1/15 [00:39<09:14, 39.59s/it]
 13% 2/15 [00:42<03:51, 17.81s/it]
 20% 3/15 [00:44<02:08, 10.73s/it]
 27% 4/15 [00:46<01:21,  7.43s/it]
 33% 5/15 [00:49<00:57,  5.71s/it]
 40% 6/15 [00:52<00:41,  4.65s/it]
 47% 7/15 [00:54<00:31,  3.89s/it]
 53% 8/15 [00:56<00:23,  3.39s/it]
 60% 9/15 [00:59<00:18,  3.06s/it]
 67% 10/15 [01:01<00:14,  2.86s/it]
 73% 11/15 [01:04<00:11,  2.77s/it]
 80% 12/15 [01:06<00:07,  2.63s/it]
 87% 13/15 [01:08<00:05,  2.54s/it]
 93% 14/15 [01:11<00:02,  2.49s/it]
100% 15/15 [01:17<00:00,  5.16s/it]
Load checkpoint from: c

In [8]:
%cd /content
!mkdir images
%cd /content/Wav2Lip
import cv2
from tqdm import tqdm
from os import path
import os

vidcap = cv2.VideoCapture("/content/Wav2Lip/results/result_voice.mp4")
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join('/content/images', str(frameNumber).zfill(4)+'.jpg'), image)

/content
mkdir: cannot create directory ‘images’: File exists
/content/Wav2Lip
FPS:  25.0 Frames:  497


100%|██████████| 497/497 [00:10<00:00, 45.54it/s]


In [9]:
%cd /content/CodeFormer

import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('CodeFormer', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

/content/CodeFormer


In [10]:
%cd /content/CodeFormer

# CODEFORMER_FIDELITY = 0.5
# !python inference_codeformer.py -w $CODEFORMER_FIDELITY --has_aligned --input_path /content/images

# whole images
CODEFORMER_FIDELITY = 0.7
!python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path /content/images --bg_upsampler realesrgan

/content/CodeFormer
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/RealESRGAN_x2plus.pth" to /content/CodeFormer/weights/realesrgan/RealESRGAN_x2plus.pth

100% 64.0M/64.0M [00:00<00:00, 136MB/s]
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth" to /content/CodeFormer/weights/CodeFormer/codeformer.pth

100% 359M/359M [00:01<00:00, 248MB/s]
Face detection model: retinaface_resnet50
Background upsampling: True, Face upsampling: False
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/CodeFormer/weights/facelib/detection_Resnet50_Final.pth

100% 104M/104M [00:00<00:00, 205MB/s] 
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/parsing_parsenet.pth" to /content/CodeFormer/weights/facelib/parsing_parsenet.pth

100% 81.4M/81.4M [00:01<00:00, 80.9MB/s]
[1/497] Processing: 0000.jpg
	detect 1 faces
[2/497] Processing: 0001.jpg
	detec

In [11]:
import os
restoredFramesPath = '/content/CodeFormer/results/images_0.7' + '/final_results/'
processedVideoOutputPath = '/content/output'

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

import cv2
import numpy as np

batch = 0
batchSize = 600
from tqdm import tqdm
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end)
  for filename in  tqdm(dir_list[start:end]):
      filename = restoredFramesPath+filename;
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)


  out = cv2.VideoWriter(processedVideoOutputPath+'/batch_'+str(batch).zfill(4)+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
  batch = batch + 1

  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()

#Generate audio video
audio = ffmpeg.input(f'/content/input/5566.mp3')
video = ffmpeg.input(f'/content/output/batch_0000.avi')

out = ffmpeg.output(video, audio, f'/content/output/final.mp4')
print("output/final.mp4")

  0%|          | 0/1 [00:00<?, ?it/s]

processing  0 600



100%|██████████| 1/1 [00:48<00:00, 48.14s/it]

output/final.mp4
